In [1]:
import pymysql
import string
import pandas as pd

In [2]:
conn = pymysql.connect(
    user ="root",
    password="password",
    host="127.0.0.1",
    port=3306,
    cursorclass=pymysql.cursors.DictCursor
)
with conn.cursor() as cursor:
    cursor.execute("DROP DATABASE IF EXISTS chats")
    data = cursor.fetchall()
    conn.commit()
    cursor.close()

In [3]:
conn = pymysql.connect(
    user ="root",
    password="password",
    host="127.0.0.1",
    port=3306,
    cursorclass=pymysql.cursors.DictCursor
)
with conn.cursor() as cursor:
    cursor.execute("CREATE DATABASE IF NOT EXISTS chats")
    data = cursor.fetchall()
    conn.commit()
    cursor.close()

In [4]:
def execute_mysql_query(query):
    conn = pymysql.connect(
        user ="root",
        password="password",
        host="127.0.0.1",
        port=3306,
        database='chats',
        cursorclass=pymysql.cursors.DictCursor
    )
    with conn.cursor() as cursor:
        cursor.execute(query)
        data = cursor.fetchall()
        conn.commit()
        cursor.close()
    return data

In [5]:
query = '''
DROP TABLE IF EXISTS user_id_channels;
DROP TABLE IF EXISTS users_org_details;
DROP TABLE IF EXISTS users;
DROP TABLE IF EXISTS users_email;
DROP TABLE IF EXISTS org;

CREATE TABLE `org` (
  `org_id` int(11) NOT NULL AUTO_INCREMENT,
  `name` text NOT NULL,
  PRIMARY KEY (`org_id`),
  UNIQUE KEY `org_name` (`name`) USING HASH
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;

CREATE TABLE `users_email` (
  `uid` int(11) NOT NULL AUTO_INCREMENT,
  `email` text NOT NULL,
  `password` text NOT NULL DEFAULT '$2a$08$dVnj.sN1OOPEWhtNmexzmuCZLodGp1u9r9nLRLAbMVivMGnbKhI2.',
  PRIMARY KEY (`uid`),
  UNIQUE KEY `user_email` (`email`) USING HASH
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;

CREATE TABLE `users_org_details` (
  `id` int(11) NOT NULL AUTO_INCREMENT,
  `org_id` int(11) NOT NULL,
  `uid` int(11) NOT NULL,
  `is_active` int(11) DEFAULT 1,
  PRIMARY KEY (`id`),
  UNIQUE KEY `users_org_details_key` (`org_id`,`uid`),
  KEY `uid` (`uid`),
  CONSTRAINT `users_org_details_ibfk_1` FOREIGN KEY (`org_id`) REFERENCES `org` (`org_id`),
  CONSTRAINT `users_org_details_ibfk_2` FOREIGN KEY (`uid`) REFERENCES `users_email` (`uid`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;

CREATE TABLE `users` (
  `uid` int(11) NOT NULL,
  `first_name` text DEFAULT NULL,
  `last_name` text DEFAULT NULL,
  `status` text DEFAULT NULL,
  `profile_picture` blob DEFAULT NULL,
  UNIQUE KEY `users_key` (`uid`),
  CONSTRAINT `users_ibfk_1` FOREIGN KEY (`uid`) REFERENCES `users_email` (`uid`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;

CREATE TABLE `user_id_channels` (
  `channel_id` int(11) NOT NULL AUTO_INCREMENT,
  `org_id` int(11) NOT NULL,
  `uid_array` text NOT NULL,
  PRIMARY KEY (`channel_id`),
  KEY `user_id_channels_ibfk_1` (`org_id`),
  CONSTRAINT `user_id_channels_ibfk_1` FOREIGN KEY (`org_id`) REFERENCES `org` (`org_id`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
'''

In [6]:
queries = query.replace("\n","").split(";")

In [7]:
for i in range(len(queries)):
    print(queries[i])
    if queries[i] != "":
        execute_mysql_query(queries[i])

DROP TABLE IF EXISTS user_id_channels
DROP TABLE IF EXISTS users_org_details
DROP TABLE IF EXISTS users
DROP TABLE IF EXISTS users_email
DROP TABLE IF EXISTS org
CREATE TABLE `org` (  `org_id` int(11) NOT NULL AUTO_INCREMENT,  `name` text NOT NULL,  PRIMARY KEY (`org_id`),  UNIQUE KEY `org_name` (`name`) USING HASH) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4
CREATE TABLE `users_email` (  `uid` int(11) NOT NULL AUTO_INCREMENT,  `email` text NOT NULL,  `password` text NOT NULL DEFAULT '$2a$08$dVnj.sN1OOPEWhtNmexzmuCZLodGp1u9r9nLRLAbMVivMGnbKhI2.',  PRIMARY KEY (`uid`),  UNIQUE KEY `user_email` (`email`) USING HASH) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4
CREATE TABLE `users_org_details` (  `id` int(11) NOT NULL AUTO_INCREMENT,  `org_id` int(11) NOT NULL,  `uid` int(11) NOT NULL,  `is_active` int(11) DEFAULT 1,  PRIMARY KEY (`id`),  UNIQUE KEY `users_org_details_key` (`org_id`,`uid`),  KEY `uid` (`uid`),  CONSTRAINT `users_org_details_ibfk_1` FOREIGN KEY (`org_id`) REFERENCES `org` (`org_id`),  

In [8]:
org = pd.read_csv("orgs.csv")

for index, row in org.iterrows():
    query = '''INSERT INTO org(org_id,name) VALUES ('%s','%s')'''%(row['org_id'],row['name'])
    execute_mysql_query(query)

In [9]:
users_email = pd.read_csv("users_email.csv")

for index, row in users_email.iterrows():
    query = '''INSERT INTO users_email(uid,email,password) VALUES ('%s','%s','%s')'''%(row['uid'],row['email'],row['password'])
    execute_mysql_query(query)

In [10]:
users_org_details = pd.read_csv("users_org_details.csv")

for index, row in users_org_details.iterrows():
    query = '''INSERT INTO users_org_details(id,org_id,uid,is_active) VALUES ('%s','%s','%s','%s')'''%(row['id'],row['org_id'],row['uid'],row['is_active'])
    execute_mysql_query(query)

In [11]:
users = pd.read_csv("users.csv")

for index, row in users.iterrows():
    query = '''INSERT INTO users(uid,first_name,last_name,status) VALUES ('%s','%s','%s','%s')'''%(row['uid'],row['first_name'],row['last_name'],row['status'])
    execute_mysql_query(query)

In [12]:
user_id_channels = pd.read_csv("user_id_channels.csv")

for index, row in user_id_channels.iterrows():
    query = '''INSERT INTO user_id_channels(channel_id,org_id,uid_array) VALUES ('%s','%s','%s')'''%(row['channel_id'],row['org_id'],row['uid_array'])
    execute_mysql_query(query)

In [13]:
query = '''
CREATE PROCEDURE `addNewUser`(IN user_mail text,IN user_password text)
BEGIN
	INSERT INTO users_email(email,password) VALUES (user_mail,user_password);
	COMMIT;
END;
$$
CREATE PROCEDURE `addOrg`(IN org_name text)
BEGIN
    INSERT INTO org(name) VALUES (org_name);
    COMMIT;
END;
$$
CREATE PROCEDURE `addUserOrgDetails`(IN orgID int(11) , IN userID int(11))
BEGIN
	IF NOT EXISTS (SELECT 1 FROM users_org_details WHERE org_id = orgID AND uid = userID and is_active = 0)
	THEN
		INSERT INTO users_org_details (org_id,uid) VALUES (orgID,userID);
	ELSE 
		UPDATE users_org_details
		SET is_active = 1
		WHERE org_id = orgID and uid = userID;
	END IF;
    
    COMMIT;
END;
$$
CREATE PROCEDURE `getChatId`(IN orgID text, IN uidArray text)
BEGIN
	DECLARE active_count INT;
	IF NOT EXISTS (SELECT 1 FROM user_id_channels WHERE org_id = orgID AND uid_array = uidArray)
    THEN
		INSERT INTO user_id_channels (org_id,uid_array) VALUES (orgID,uidArray);
	END IF;
    
    COMMIT;
    
	DROP TEMPORARY TABLE IF EXISTS temp;
	CREATE TEMPORARY TABLE temp AS
	select SUBSTRING_INDEX(uidArray,",",1) as users
	UNION
	SELECT SUBSTRING_INDEX(uidArray,",",-1) as users;

	SELECT SUM(is_active) into active_count
	FROM users_org_details uo
	JOIN temp t ON uo.uid = t.users 
	WHERE org_id = orgID AND is_active = 1;
    
    SELECT channel_id,org_id,uid_array
    FROM user_id_channels
    WHERE org_id = orgID AND uid_array = uidArray AND active_count = 2;
END;
$$
CREATE PROCEDURE `getOrg`()
BEGIN
    SELECT org_id,name FROM org;
END;
$$
CREATE PROCEDURE `getOrgLevelUsers`(IN orgID text)
BEGIN
	SELECT org_id,GROUP_CONCAT(uo.uid) uid,GROUP_CONCAT(coalesce(first_name,email)) first_name
	FROM users_org_details uo
    LEFT JOIN users u ON uo.uid = u.uid
    LEFT JOIN users_email ue ON ue.uid = uo.uid
    WHERE org_id = orgID and is_active = 1
	GROUP BY org_id;
END;
$$
CREATE PROCEDURE `getUser`(IN user_mail text)
BEGIN
	SELECT uid,email,`password`
    FROM users_email 
    WHERE email = user_mail;
END;
$$
CREATE PROCEDURE `getUserOrgDetails`(IN userID int(11))
BEGIN
	SELECT uid,GROUP_CONCAT(uo.org_id) as org_id,GROUP_CONCAT(o.name) as `name`
	FROM users_org_details uo
    JOIN org o ON uo.org_id = o.org_id
	WHERE uid = userID and is_active = 1
	GROUP BY uid;
END;
$$
CREATE PROCEDURE `getUserProfile`(IN var_uid text)
BEGIN
	SELECT uid,first_name,last_name,status,profile_picture
    FROM users
    WHERE uid = var_uid;
END;
$$
CREATE PROCEDURE `update_user_org_details`(IN var_org text,IN var_uid text,IN var_active text)
BEGIN
	UPDATE users_org_details
	SET is_active = var_active
	WHERE org_id = var_org and uid = var_uid;
    
    COMMIT;
END;
$$
CREATE PROCEDURE `updateUserProfile`(IN var_uid text,IN var_first_name text,IN var_last_name text,IN var_status text,IN var_profile_picture blob)
BEGIN
	IF NOT EXISTS (SELECT 1 FROM users WHERE uid = var_uid)
    THEN
		INSERT INTO users (uid,first_name,last_name,status,profile_picture) VALUES
				(var_uid,var_first_name,var_last_name,var_status,var_profile_picture);
	ELSE
		UPDATE users
		SET first_name = var_first_name,last_name=var_last_name,status=var_status,profile_picture = var_profile_picture
		WHERE uid = var_uid;
	END IF;
    
    COMMIT;
END;
'''

In [14]:
queries = query.replace("\n","").split("$$")

In [15]:
for i in range(len(queries)):
    print(queries[i])
    if queries[i] != "":
        execute_mysql_query(queries[i])

CREATE PROCEDURE `addNewUser`(IN user_mail text,IN user_password text)BEGIN	INSERT INTO users_email(email,password) VALUES (user_mail,user_password);	COMMIT;END;
CREATE PROCEDURE `addOrg`(IN org_name text)BEGIN    INSERT INTO org(name) VALUES (org_name);    COMMIT;END;
CREATE PROCEDURE `addUserOrgDetails`(IN orgID int(11) , IN userID int(11))BEGIN	IF NOT EXISTS (SELECT 1 FROM users_org_details WHERE org_id = orgID AND uid = userID and is_active = 0)	THEN		INSERT INTO users_org_details (org_id,uid) VALUES (orgID,userID);	ELSE 		UPDATE users_org_details		SET is_active = 1		WHERE org_id = orgID and uid = userID;	END IF;        COMMIT;END;
CREATE PROCEDURE `getChatId`(IN orgID text, IN uidArray text)BEGIN	DECLARE active_count INT;	IF NOT EXISTS (SELECT 1 FROM user_id_channels WHERE org_id = orgID AND uid_array = uidArray)    THEN		INSERT INTO user_id_channels (org_id,uid_array) VALUES (orgID,uidArray);	END IF;        COMMIT;    	DROP TEMPORARY TABLE IF EXISTS temp;	CREATE TEMPORARY TABLE t